##python学习网站 
>https://www.learnpython.org/

#棒球数据分析实战

##提出问题  
##**问题** 
>1.出生月份是否与成为明星球员有相关性？（冰球）  
>2.身体条件（BMI标准体重）与纯长打率（衡量长打能力的指标）的相关性  



In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

##**表格字段**  
###beating_df  
playerID —— 球员ID  
yearID —— 当年年份  
stint —— 球员在同一赛季的服役顺序  
G —— 比赛场数（Games）  
AB —— 打数（At Bats）打者完成一次打击，且这次打击并非是成功的牺牲触击、保送、高飞牺牲打的次数  
R —— 得分（Runs）合法及安全回到本垒的次数  
H —— 安打（Hits）在没有失误的情况下，打者把投手投出来的球，击出到界内，使打者本身能至少安全上到一垒  
2B —— 二垒（Doubles）能使打者在没有守备失误的情况下，安全到达二垒的打击  
3B —— 三垒（Triples）能使打者在没有守备失误的情况下，安全到达三垒的打击  
HR —— 本垒打（Homeruns）在没有失误的情况下，跑者一次跑完四个垒包  

###players_df  
playerID —— 球员ID  
birthYear —— 出生年  
birthMonth —— 出生月  
birthDay —— 出生日  
weight —— 体重（单位：磅 pound）  
height —— 身高（单位：英寸 inch）  
debut —— 在大联盟的首次出场日期  
finalGame —— 在大联盟的最后一场比赛日期（如果仍在活跃则为空）  

###awards_players_df  
playerID —— 球员ID  
awardID —— 奖项ID  
yearID —— 年份  

In [ ]:
beating_df  = pd.read_csv("Batting.csv")
players_df  = pd.read_csv("Master.csv")
awards_players_df = pd.read_csv("AwardsPlayers.csv")

In [ ]:
#问题1.出生月份是否与成为明星球员有相关性？
#合并获奖球员信息和球员信息，获取每个球员的出生月份
award_process = pd.merge(awards_players_df.loc[:,['playerID','awardID']], players_df.loc[:,['playerID','birthMonth']], 
                         on=['playerID'], how='right')

#填充缺失值为 0
players_type = award_process.fillna({'birthMonth':0})

#取出出生月份不等于 0 的数据，根据出生月份将数据分组，计数
players_type = players_type[players_type['birthMonth'] != 0]
players_grouped = players_type.groupby(['birthMonth']).agg(['count'])

In [ ]:
#分析1
#所有球员的生日月份分布，绿色线是每个出生月份的球员平均值
p1 = plt.bar(players_grouped.index, players_grouped[('playerID', 'count')])
plt.axhline(y=players_grouped[('playerID', 'count')].mean(), color = '#23e342')
plt.show()

In [ ]:
#获奖球员的生日月份分布，绿色线是每个出生月份的获奖球员平均值
p2 = plt.bar(players_grouped.index, players_grouped[('awardID', 'count')])
plt.axhline(y=players_grouped[('awardID', 'count')].mean(), color = '#23e342')
plt.show()

In [ ]:
#每个出生月，获奖球员占总球员比例分布，绿色线为获奖球员占全体球员的比例
(players_grouped[('awardID', 'count')]/players_grouped[('playerID', 'count')]).plot(kind='bar')
plt.axhline(y=(players_grouped[('awardID', 'count')].sum()*1.0)/(players_grouped[('playerID', 'count')].sum()*1.0), color = '#23e342')

In [ ]:
#问题2：身体条件（BMI标准体重）与纯长打率（衡量长打能力的指标）的相关性
#累加每个球员的所有能力指标数据备用
beating_select = beating_df[['playerID','AB','H','2B','3B','HR']]
beating_grouped = beating_select.groupby(['playerID'], as_index = False).sum()

#AB打数太少，计算纯长打率的价值不大，截取AB大于100的数据进行研究
beating_grouped = beating_grouped[beating_grouped['AB'] > 100]

#按公式进行向量计算，求IsoP
beating_grouped["IsoP"] = (beating_grouped['H'] + beating_grouped['2B'] + (2 * beating_grouped['3B']) + (3 * beating_grouped['HR'])) / beating_grouped['AB'] - beating_grouped['H'] / beating_grouped['AB'] 

#截取IsoP大于0的数据进行分析并查看各项描述统计学数据
beating_grouped = beating_grouped[beating_grouped['IsoP'] > 0]

#选择球员数据中的身高和体重，填充空值为0
players_select = players_df[['playerID','weight','height']].fillna(0)

#选择身高和体重数据大于0的球员，转换身高和体重的单位
players_select = players_select[(players_select['weight']>0) & (players_select['height']>0)]
players_select['weight_kg'] = players_select['weight'] * 0.4535924
players_select['height_m'] = players_select['height'] * 0.0254
del players_select['weight']
del players_select['height']

#按照公式求 BMI
players_select['BMI'] = players_select['weight_kg'] / (players_select['height_m'] * players_select['height_m'])

#BMI值直方图分布
sns.distplot(players_select['BMI'])

In [ ]:
#合并以上两个处理后的数据集，选取需要的数据项
beating_players_df = pd.merge(beating_grouped.loc[:,['playerID','IsoP']], players_select.loc[:,['playerID','BMI']], 
                              on=['playerID'], how='inner')

# 按照(18.5,23],(23,25],(25,27],(27,30],(30,40]分类，对应名称'Normal_1','Normal_2','Overweight_1','Overweight_2','Obese'
# 对应标准1、标准2、超重1、超重2、肥胖（对于运动员来说应该是强壮，以下用强壮代指Obese阶段）
bmi_array = beating_players_df['BMI']
bins = [18.5,23,25,27,30,40]
bmi_level = ['Normal_1','Normal_2','Overweight_1','Overweight_2','Obese']
beating_players_df['BMI_level'] = pd.cut(beating_players_df['BMI'], bins,labels = bmi_level)

In [ ]:
#滤除缺失数据
beating_players_df = beating_players_df.dropna()

In [ ]:
#分析2
# 直接通过点状图查看 BMI 值和 IsoP 值的相关性
plt.scatter(beating_players_df['BMI'], beating_players_df['IsoP'])

In [ ]:
#每个BMI_level的IsoP的均值分布
sns.barplot(beating_players_df['BMI_level'], beating_players_df['IsoP'], palette="Blues_d")

In [ ]:
#每个BMI_level的IsoP的箱线图比较
sns.boxplot(x="BMI_level", y="IsoP", data=beating_players_df, palette="Blues_d", width=0.8)

In [ ]:
#按照BMI值的分组标准，计算每个分组内的各项统计学数据
isop_bmi_togroup = beating_players_df[['IsoP','BMI_level']]
functions = ['count','mean','median','max','min']
isop_bmi_grouped = isop_bmi_togroup.groupby(['BMI_level']).agg(functions)
isop_bmi_grouped

In [ ]:
#相关性分析
np.corrcoef(beating_players_df['BMI'], beating_players_df['IsoP'])